### Extract Lyrics Embeddings

In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
import scipy as sp

/Users/chloe/opt/miniconda3/envs/BKMS/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import dataset
data = pd.read_csv('./spotify_data.csv', index_col=[0])
lyrics_data = pd.DataFrame(data['lyrics'])
lyrics_data['embedding'] = ''
lyrics_data['lyrics'] = lyrics_data['lyrics'].apply(lambda e: e.replace('"',''))

In [38]:
a = 'Hi there! "hehehe"'
a = a.replace('"', '')
print(a)

Hi there! hehehe


In [11]:
lyrics_data

,lyrics,embedding
160,Aah Aah The games you played were never fun Yo...,
445,The lights go out and I can't be saved Tides t...,
763,"'Cause you're a sky, 'cause you're a sky full ...",
856,I will not make the same mistakes that you did...,
1169,Heyy Heeey Heey Your lipstick stains On the fr...,
...,...,...
16203,"Yeah, you! Yeah, you! I used to wanna be Livin...",
16284,"If you don't wanna see me Did a full 180, craz...",
16771,"Nah, nah, nah Cake by the ocean Oh, no See you...",
17046,"Come on, come on, turn the radio on It's Frida...",


In [35]:
# define model
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig, pipeline
import torch

config = RobertaConfig.from_pretrained("SamLowe/roberta-base-go_emotions")
config.output_hidden_states = True
tok = RobertaTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions")
model = RobertaModel.from_pretrained("SamLowe/roberta-base-go_emotions", config=config)

# extract embedding
print(lyrics_data['lyrics'].iloc[2])
print(model(torch.tensor([tok.encode(lyrics_data['lyrics'].iloc[3])])).pooler_output.squeeze().detach().shape)
# model(torch.tensor([tok.encode(lyrics_data['lyrics'].iloc[1])])).pooler_output.squeeze()
for idx in range(len(lyrics_data)):
    element = model(torch.tensor([tok.encode(lyrics_data['lyrics'].iloc[idx])])).pooler_output.squeeze().detach()
    print(idx, element.shape)
    lyrics_data['embedding'].iloc[idx] = element
# lyrics_data['embedding'] = lyrics_data['lyrics'].apply(lambda e: model(torch.tensor([tok.encode(e)])).pooler_output.squeeze())
lyrics_data

Some weights of RobertaModel were not initialized from the model checkpoint at SamLowe/roberta-base-go_emotions and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'Cause you're a sky, 'cause you're a sky full of stars I'm gonna give you my heart 'Cause you're a sky, 'cause you're a sky full of stars 'Cause you light up the path I don't care, go on and tear me apart I don't care if you do, ooh-ooh, ooh'Cause in a sky, 'cause in a sky full of stars I think I saw you 'Cause you're a sky, 'cause you're a sky full of stars I wanna die in your arms, oh, oh, oh'Cause you get lighter the more it gets dark I'm gonna give you my heart, oh I don't care, go on and tear me apart I don't care if you do, ooh-ooh, ooh'Cause in a sky, 'cause in a sky full of stars I think I see you I think I see you 'Cause you're a sky, you're a sky full of stars Such a heavenly view You're such a heavenly view (Yeah, yeah, yeah, ooh) 'Cause you're a sky, 'cause you're a sky full of stars I'm gonna give you my heart 'Cause you're a sky, 'cause you're a sky full of stars 'Cause you light up the path I don't care, go on and tear me apart I don't care if you do, ooh-ooh, ooh'Cause 

Token indices sequence length is longer than the specified maximum sequence length for this model (674 > 512). Running this sequence through the model will result in indexing errors


0 torch.Size([768])
1 torch.Size([768])


RuntimeError: The expanded size of the tensor (674) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 674].  Tensor sizes: [1, 514]

In [ ]:
import pickle
with open('./lyrics_embedding.pkl', 'wb') as f:
    pickle.dump(lyrics_data, f)